## Converting Model from Pytorch file to Tensor Flow js file

### Converting model to ONNX format

In [ ]:
import torch
from efficientnet_pytorch import EfficientNet
import torch.onnx
import onnx

# Load the model with the correct number of output classes
model_name = 'efficientnet-b0'  # Adjust based on the specific EfficientNet variant you used
num_classes = 10  # The number of classes in your dataset
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Load the state dictionary
checkpoint_path = '/Users/mrinoyb2/git/Wound-Detection/Models/efficientnet_model_10_epochs.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)

model.eval()  # Set the model to evaluation mode

# Input to the model
dummy_input = torch.randn(1, 3, 224, 224)  # Adjust the size if necessary for EfficientNet

# Export the model to ONNX
onnx_path = "/Users/mrinoyb2/git/Wound-Detection/Models/efficientnet_model_10_epochs.onnx"
torch.onnx.export(model, dummy_input, onnx_path,
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names=['input'],
                  output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

print('Model exported to ONNX format.')

# Load and check the ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

print('Model is well formed.')


### Installing necessary libraries

%pip install onnx tensorflow onnx-tf
%pip install tensorflow_probability
%pip install tf2onnx


### Convert Model from ONNX to Tensorflow

In [ ]:
# convert onnx to tensorflow
import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("/Users/mrinoyb2/git/Wound-Detection/Models/wound_model_50_epochs.onnx")

# Prepare the ONNX model for inference
tf_rep = prepare(onnx_model)

# Export the model to TensorFlow format using onnx-tf
tf_rep.export_graph("/Users/mrinoyb2/git/Wound-Detection/Models/wound_model_50_epochs.pb")

print('Model exported to TensorFlow format.')

### Install libraries to convert tf to tfjs

In [ ]:
!npm install -g @tensorflow/tfjs-node

!tensorflowjs_converter --input_format=tf_saved_model --output_node_names='efficient_net2' --control_flow_v2=true /Users/mrinoyb2/git/Wound-Detection/Models/efficientnet_model_10_epochs.tf /Users/mrinoyb2/git/Wound-Detection/Models/efficientnet2_model_10_epochs_tfjs